In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define image size and path to dataset
IMG_SIZE = 64
DATASET_PATH = 'path/to/dataset'

# Load and preprocess images
def load_images():
    X = []
    y = []
    for label in os.listdir(DATASET_PATH):
        for file in os.listdir(os.path.join(DATASET_PATH, label)):
            img_path = os.path.join(DATASET_PATH, label, file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# Split dataset into training and testing sets
X, y = load_images()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile and train model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate model on testing set
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

# Save trained model to disk
model.save('path/to/trained/model.h5')


In [1]:
#Here is an example Python code for detecting hand gestures live using the trained CNN model:

import cv2
import numpy as np
from keras.models import load_model

# Define image size and path to trained model
IMG_SIZE = 64
MODEL_PATH = 'path/to/trained/model.h5'

# Load trained model
model = load_model(MODEL_PATH)

# Define class labels
labels = ['Fist', 'Five', 'Four', 'Ok', 'Three', 'Two']

# Define video capture device
cap = cv2.VideoCapture(0)

while True:
    # Capture frame from video stream
    ret, frame = cap.read()
    
    # Preprocess image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    
    # Make prediction using trained model
    pred = model.predict(img)
    label = labels[np.argmax(pred)]
    
    # Draw label on frame
    cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display frame
    cv2.imshow('Hand Gesture Recognition', frame)
    
    # Exit loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture device and close window
cap.release()
cv2.destroyAllWindows()
